## Web Scraping : Mubawab.ma


In [2]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,108 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,616 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,164 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com

In [3]:
import numpy as np
import pandas as pd

### Get selenium webdriver



In [4]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException # import the required exception

### function to scrape each page


In [ ]:
# let's define a function that takes each page of listings and scrap it
search_link = "https://www.mubawab.ma/fr/st/casablanca/appartements-a-vendre:p:"
prices = []
quartiers = []
standings = []
ids = []
def get_data(link, prices, quartiers, standings, ids):
  wd.get(link)
  n_elems = len(wd.find_elements(By.XPATH, '//*[@class="ulListing"]/li'))
  for i in range(n_elems):
    # price
    prix = wd.find_element(By.XPATH, f'//*[@class="ulListing"]/li[{i+1}]/div[1]/div[1]/span') #Locate the price
    prix_text = prix.text #Get the price value
    prices.append(prix_text)

    # quartier
    quartier = wd.find_element(By.XPATH, f'//*[@class="ulListing"]/li[{i+1}]/div[2]/h3') #Locate the nq'th neighbourhood to select
    qrt_text = quartier.text #Get the neighbourhood name
    quartiers.append(qrt_text)

    # specs: chambre, surface
    try:
      specs = wd.find_element(By.XPATH, f'//*[@class="ulListing"]/li[{i+1}]/div[2]/div[1]/h4') #Locate the nq'th standing to select
      specs_text = specs.text #Get the standing_text name
    except NoSuchElementException:
      specs_text = ''
    standings.append(specs_text)

    # link_annonce
    id_annonceur = wd.find_element(By.XPATH, f'//*[@class="ulListing"]/li[{i+1}]')
    promotion_id = id_annonceur.get_attribute("linkref")
    ids.append(promotion_id)


### scrape all the pages of this search link
there's 161 pages for the search in Casablanca

In [1]:
# WARNING !! it could take nearly 15min to execute
wd = webdriver.Chrome(options=options)
# execute the function multiple times
for i in range(161):
  link = search_link + str(i+1)
  get_data(link, prices, quartiers, standings, ids)

NameError: name 'webdriver' is not defined

### Prepare a dataframe that has all findings
it would create an initial excel sheet that has details of the annonce (if available) from the search page.

In [ ]:
all_data = [prices , quartiers , standings , ids  ]
df = pd.concat([pd.DataFrame(lst) for lst in all_data] , axis=1).reset_index(drop = True)
#Set the dataframe's columns
df.columns = ['prices','quartiers','spec','ids']
df
df.to_excel(r'mubawab_reduced_data.xlsx', index = False)

### Get Latitude , Longitude and other details of the annonce
we get the webpage of each annonce and get all details possible. an store them in the same dataframe from earlier.


In [ ]:
# WARNING !! it would take several hours to execute
lats = []
longs = []
surfaces = []
nbrs_pieces = []
nbrs_chambres = []
nbrs_bains = []
descriptions = []
# other features

types_biens = []
etats = []
etages = []
orientations = []
type_sols = []
standings = []
etats_constructions = []
Livraisons = []
all_keys_correspondant_array = {'Type de bien':types_biens, 'Etat':etats, 'Étage du bien':etages, 'Orientation':orientations, 'Type du sol': type_sols, 'Standing':standings, 'État':etats_constructions, 'Livraison':Livraisons}


import time
t = 0.1
# how much rows in the df
n_elems = len(df)

for j in range(n_elems):
    # long and lat
    annonce_link = df.iloc[j,3]
    wd.get(annonce_link)
    try:
      mapOpen = wd.find_element(By.ID,'mapOpen')
      lat = mapOpen.get_attribute("lat")
      lon = mapOpen.get_attribute("lon")
    except NoSuchElementException:
      lat = ''
      lon = ''
    lats.append(lat)
    longs.append(lon)
    # get caracteristics
    ## get surface
    try:
      surface = wd.find_element(By.XPATH, f'//*[@class="col-8"]/div[1]/div[3]/div[1]/span')
      surface_text = surface.text
    except NoSuchElementException:
      surface_text = ''
    surfaces.append(surface_text)
    ## get nbr_piece
    try:
      nbr_piece = wd.find_element(By.XPATH, f'//*[@class="col-8"]/div[1]/div[3]/div[2]/span')
      nbr_piece_text = nbr_piece.text
    except NoSuchElementException:
      nbr_piece_text = ''
    if "Pièces" in nbr_piece_text:
      nbrs_pieces.append(nbr_piece_text)
    elif "Chambres" in nbr_piece_text:
      nbrs_chambres.append(nbr_piece_text)
    elif "Salle de bain" in nbr_piece_text:
      nbrs_bains.append(nbr_piece_text)
    else:
      nbrs_pieces.append(nbr_piece_text)
    # get nbr_chambre
    try:
      nbr_chambre = wd.find_element(By.XPATH, f'//*[@class="col-8"]/div[1]/div[3]/div[3]/span')
      nbr_chambre_text = nbr_chambre.text
    except NoSuchElementException:
      nbr_chambre_text = ''
    nbrs_chambres.append(nbr_chambre_text)
    if "Pièces" in nbr_piece_text:
      nbrs_pieces.append(nbr_piece_text)
    elif "Chambres" in nbr_piece_text:
      nbrs_chambres.append(nbr_piece_text)
    elif "Salle de bain" in nbr_piece_text:
      nbrs_bains.append(nbr_piece_text)
    else:
      nbrs_pieces.append(nbr_piece_text)
    # get nbr_bain
    try:
      nbr_bain = wd.find_element(By.XPATH, f'//*[@class="col-8"]/div[1]/div[3]/div[4]/span')
      nbr_bain_text = nbr_bain.text
    except NoSuchElementException:
      nbr_bain_text = ''
    nbrs_bains.append(nbr_bain_text)
    if "Pièces" in nbr_piece_text:
      nbrs_pieces.append(nbr_piece_text)
    elif "Chambres" in nbr_piece_text:
      nbrs_chambres.append(nbr_piece_text)
    elif "Salle de bain" in nbr_piece_text:
      nbrs_bains.append(nbr_piece_text)
    else:
      nbrs_pieces.append(nbr_piece_text)
    # get description
    try:
      description = wd.find_element(By.XPATH, f'//*[@class="col-8"]/div[4]/p')
      description_text = description.text
    except NoSuchElementException:
      description_text = ''
    descriptions.append(description_text)
    # get etat
    try:
      etat = wd.find_element(By.XPATH, f'//*[@class="col-8"]/div[6]/div[1]/div[2]/div[2]/p[2]')
      etat_text = etat.text
    except NoSuchElementException:
      etat_text = ''
    etats.append(etat_text)
    # get other specs
    try:
      spec = wd.find_element(By.CLASS_NAME,'adFeatures')
      spec_text = spec.text
      #---- parsing the text of spec_text -------
      ## Split the text into lines
      lines = spec.text.strip().split('\n')
    except NoSuchElementException:
      spec_text = ''
      lines = []
    ## Create a dictionary to store the key-value pairs
    data = {}
    print(f"{j}: {', '.join(lines)}")
    ## Iterate through the lines, taking two at a time
    for i in range(0, len(lines), 2):
      key = lines[i].strip()
      value = lines[i + 1].strip()
      data[key] = value
    ## appending in lists
    for k in all_keys_correspondant_array.keys():
      try:
        v1 = data[k]
      except KeyError:
        v1 = ''
      all_keys_correspondant_array[k].append(v1)




all_data = [prices , quartiers , standings , ids , lats , longs, surfaces, nbrs_pieces, nbrs_chambres, nbrs_bains,
descriptions, types_biens, etats, etages, orientations, type_sols, standings, etats_constructions, Livraisons]
df = pd.concat([pd.DataFrame(lst) for lst in all_data], axis=1).reset_index(drop = True)
#Set the dataframe's columns
df.columns = ['prices','quartiers','spec','ids','lats','longs','surfaces', 'nbrs_pieces', 'nbrs_chambres', 'nbrs_bains',
'descriptions', 'types_biens', 'etats', 'etages', 'orientations', 'type_sols', 'standings', 'etats_constructions', 'Livraisons']

Streaming output truncated to the last 5000 lines.
452: Type de bien, Appartement, Étage du bien, 5ème
453: Type de bien, Appartement, Étage du bien, 3ème
454: Type de bien, Appartement, Étage du bien, 3ème
455: Type de bien, Appartement, Étage du bien, 3ème
456: Type de bien, Appartement
457: Type de bien, Appartement, Étage du bien, 1er
458: Type de bien, Appartement, Étage du bien, 2ème
459: Type de bien, Appartement, Étage du bien, 5ème
460: Type de bien, Appartement, Étage du bien, 1er
461: Type de bien, Appartement, Étage du bien, 5ème
462: Type de bien, Appartement
463: Type de bien, Appartement, Étage du bien, 2ème
464: Type de bien, Appartement, Étage du bien, 1er
465: Type de bien, Appartement, Étage du bien, 6ème
466: Type de bien, Appartement, Etat, Nouveau, Standing, Haut standing, État, En cours de construction, Livraison:, mai 2026
467: Type de bien, Appartement, Etat, Nouveau, Standing, Moyen standing, État, Finalisé
468: Type de bien, Appartement, Etat, Nouveau, Standi

### Show all results in a Dataframe

In [ ]:
df

,prices,quartiers,spec,ids,lats,longs,surfaces,nbrs_pieces,nbrs_chambres,nbrs_bains,descriptions,types_biens,etats,etages,orientations,type_sols,standings,etats_constructions,Livraisons
0,935 000 DH,Bernoussi à Casablanca,Moyen standing,https://www.mubawab.ma/fr/pa/7284454/apartemen...,33.61040882133904,-7.480355685654416,99 m²,4 Pièces,3 Chambres,2 Salles de bains,"Avec plus de 35 ans d’expérience, le groupe Ik...",Appartement,Nouveau,,,,Moyen standing,Finalisé,
1,1 866 700 DH,Californie à Casablanca,Haut standing,https://www.mubawab.ma/fr/pa/7902993/superbe-a...,33.52868056914214,-7.613258058098552,113 m²,4 Pièces,3 Chambres,2 Salles de bains,"Les Allées de Californie 2, plus qu’une simple...",Appartement,Nouveau,,,,Haut standing,Finalisé,
2,1 575 000 DH,Belvédère à Casablanca,Haut standing,https://www.mubawab.ma/fr/pa/7461573/apparteme...,33.58529858553826,-7.598934731436941,122 m²,4 Pièces,3 Chambres,2 Salles de bains,CHRAIBI IMMOBILIER est un projet résidentiel c...,Appartement,Nouveau,2ème,Sud,Marbre,Haut standing,Finalisé,
3,1 150 000 DH,Belvédère à Casablanca,Haut standing,https://www.mubawab.ma/fr/pa/7461580/apparteme...,33.58529858553826,-7.598934731436941,99 m²,4 Pièces,2 Chambres,2 Salles de bains,CHRAIBI IMMOBILIER est un projet résidentiel c...,Appartement,Nouveau,5ème,Sud,Marbre,Haut standing,Finalisé,
4,1 175 640 DH,Aïn Sebaâ à Casablanca,Moyen standing,https://www.mubawab.ma/fr/pa/7885304/charmant-...,33.60439330562992,-7.541969962385935,96 m²,4 Pièces,3 Chambres,2 Salles de bains,Charmant appartement de 71 m² composé de 2 cha...,Appartement,Nouveau,,,,Moyen standing,Finalisé,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 Pièces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 Pièces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 Pièces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 Pièces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save Data in xlsx file

In [ ]:
df.to_excel(r'mubawab_data.xlsx', index = False)